# Test Spherical Signal Generation

## Description
This notebook provides a test script to generate the spherical features for input. 

In [8]:
import sys
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import open3d as o3d

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the test features

In [48]:
filename = '/home/berlukas/Documents/workspace/phaser_ws/src/s2loc/script/playground/test_features.npy'
dense_map_filename = '/home/berlukas/Documents/workspace/phaser_ws/src/s2loc/script/playground/foo.npy'
features = np.load(filename)
dense_map = np.load(dense_map_filename)
print(f'Features have the shape of {features.shape}')
print(f'Dense map has the shape of {dense_map.shape}')

Features have the shape of (3, 2, 200, 200)
Dense map has the shape of (446248, 4)


In [127]:
def create_grid(bw):
        n_grid = 2 * bw
        k = 0;
        points = np.empty([n_grid * n_grid, 2])
        for i in range(n_grid):
            for j in range(n_grid):
                points[k, 0] = (np.pi*(2*i+1))/(4*bw)
                points[k, 1] = (2*np.pi*j)/(2*bw);
                k = k + 1;
        return points
    
def convert_spherical_to_euclidean(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])
    return cart_grid

def map_intensity_to_RGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))
    
def sampling_cloud_from_bw(bw):
    grid = create_grid(bw)
    xyz_grid = convert_spherical_to_euclidean(grid)
    intensities = np.zeros((xyz_grid.shape[0],1))
    sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
    return sampling_grid

def create_pc(cloud, bw = 100):    
    points_xyz = cloud[:,0:3]        
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_xyz[:, 0:3])
    colors = map_intensity_to_RGB(cloud[:, 3])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    return pcd   

def create_spherical_pc(feature, bw = 100):
    pc = sampling_cloud_from_bw(bw)    
    points_xyz = pc[:,0:3]    
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_xyz[:, 0:3])
    colors = map_intensity_to_RGB(feature[:, 1])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])  
    return pcd   

def visualize_pointcloud(cloud):
    pcd = create_pc(cloud)
    o3d.visualization.draw_geometries([pcd])
    
def visualize_feature(feature, bw = 100):
    pcd = create_spherical_pc(feature, bw)    
    o3d.visualization.draw_geometries([pcd])  

In [129]:
n_features = features.shape[0]
n_features = 1
for i in range(0, n_features):
    cur_features = features[i, :, :, :] # features, 2*bw, 2*bw
    cur_features = np.reshape(cur_features, (cur_features.shape[0], -1)).T # (2*bw)^2, features    
    visualize_feature(cur_features)    

In [122]:
visualize_pointcloud(dense_map)